# Demo-notebook for bruk av funksjoner til å regne ut energiavgifter tilknyttet fossilt brensel

*Forfatter: Benedikt Goodman*

I denne teksten vil kun essensen av hva hvert objekt og metode gjør beskrives. Dersom detaljert informasjon om et objekt trenges kan man skrive **help(objektnavn.metodenavn)** eller **help(funksjonsnavn)** for å få tak i informasjonen om hva objektene/funksjonene gjør og hva de godtar av input.

In [1]:
# Import main libraries
import pandas as pd
import numpy as np
import os

# Change directory until find project root
notebook_path = os.getcwd()
while "pyproject.toml" not in os.listdir():
    os.chdir("../")

# Import math modules
from src.functions.main_program_functions import divider, multiplier, proportion_func, make_sum_column, diff_maker
# Import data utils
from src.functions.main_program_functions import merge_func, subsetter, multi_subsetter, filename_removal_func, associate_codes, rounding_error_dealer, column_tidy_func
# Import verifier functions
from src.functions.main_program_functions import data_integrity_checker, unntak_filler

# Import helperfunction for making mineral oils
from src.functions.main_program_functions import calculate_mineral_oil

# Import data import assets
from src.functions.import_class import DataImporter
from src.functions.metadata_generator import FolderSearcher

# Import export helpers
from src.functions.export_helpers import batch_exporter

# Reset current working directory after local imports
os.chdir(notebook_path)

## Om objektene som er laget for dette produksjonsløpet

### Identifikasjon av data ved hjelp av FolderSearcher

#### Instansiering av objektet
Brukeren navigerer til filstien (path) hvor dataene for beregningsopplegget ligger. Deretter definerer hen en liste med datasett (dataset_list) som skal importereres. Denne listen brukes av FolderSearcher som leter etter mapper på lokasjonen til path som matcher listen. Brukeren kan også anngi datatypen objektet skal lete etter (datatype).

FolderSearcher spør også om start og sluttperiode for import av data. For at denne funksjonen skal fungere må folderne den importerer data fra har år som navn. Objektet tar ikke hensyn til om det er år som mangler i filhirearkiet.

#### Om metodene til FolderSearcher
- *.generate_metadata()* lager FolderSearcher en dataframe filstier, filnavn og hvilken i hvilken mappe filene finnes 
- *.subset_datasets()* beholder kun verdiene (keep_vals=[]) i angitt gruppe (groups=[]). I dette tilfellet fjerner den alle forekomster av energiregnskapet unntatt 2021 fra metadataene. Disse må være lagret i lister, ellers gir objektet feilemelding. Dersom filtreringen ikke inntreffer kan det være fordi keep_vals argumentet er av feil datatype. Filtreringen godtar nemlig strings, ints, floats og det meste annet.
- *.output_df()* spytter ut dataframen objektet har laget.

Ved å dele opp innlesningen av filinformasjon så kan brukeren selv velge hvilke filer som skal importeres da alle pandas sine funksjoner fungerer på dataframen man får ut. Dermed kan man velge ut subsets av data man vil importere dersom man ser at FileSearcher har vært for generøs med hva den har funnet.

In [2]:
## Definer filsti hvor subfoldere med data finnes
path = '/ssb/stamme02/nasjregn' + '/miljoregnskaper/skatter/arkiv/'

# Skriv hvilke datasett du vil at objektet skal lete etter
datasets = [
    'innbetalte_avgifter',
    'kvotepliktige',
    'omkoding_nr',
    'satser',
    'unntakskatalog',
    'energiregnskapet']

# FolderSearcher lager en dataframe basert på input og hva den funner.
# I denne dataframen finnes alt som trengs av DataImporter for å importere data
# og kategorisere dem. Dette er en helt normal dataframe så brukeren kan endre
# på og filtrere som hen har lyst til
metadata_df = (FolderSearcher(path, datatype='sas7bdat', dataset_list=datasets)
               .generate_metadata()
               .subset_datasets(groups=['energiregnskapet'], keep_vals=['2021'])
               .output_df()
              )

Start year for data import: 2010
Last year for data import: 2021


,path,filename,directory
66,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,energiregnskapet_2021.sas7bdat,2021
13,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2010.sas7bdat,2010
19,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2011.sas7bdat,2011
25,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2012.sas7bdat,2012
31,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2013.sas7bdat,2013
37,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2014.sas7bdat,2014
43,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2015.sas7bdat,2015
55,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2016.sas7bdat,2016
49,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2017.sas7bdat,2017
1,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2018.sas7bdat,2018


In [16]:
# Vis dataframe
metadata_df

,path,filename,directory
66,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,energiregnskapet_2021.sas7bdat,2021
13,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2010.sas7bdat,2010
19,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2011.sas7bdat,2011
25,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2012.sas7bdat,2012
31,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,innbetalte_avgifter_2013.sas7bdat,2013
...,...,...,...
53,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,unntakskatalog_2017.sas7bdat,2017
5,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,unntakskatalog_2018.sas7bdat,2018
65,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,unntakskatalog_2019.sas7bdat,2019
11,/ssb/stamme02/nasjregn/miljoregnskaper/skatter...,unntakskatalog_2020.sas7bdat,2020


Alle tilgjengelige metoder har medhørende dokumentasjon som er lett tilgjengelig via pythons help() funksjonalitet.

In [3]:
# Hvordan vise dokumentasjonen til et objekt
help(FolderSearcher)

Help on function subset_datasets in module src.functions.metadata_generator:

subset_datasets(self, groups: list = None, keep_vals: list = None, group_col: 'list or str' = 'filename', val_col: 'list or str' = 'directory')
    Keep only given subsets of values from a group of data in
    a dataframe.
    
    I.e. if group_cop = filename and val_col = directory the subsetter
    will filter out values not specified in keep_vals list. This allows
    for filtering away certain years for a given datatype when making 
    the metadata_df.
    
    Parameters
    ----------
    groups : list
        Groups of data to preform subsetting on. The default is None.
    keep_vals : list
        List of values to keep within subset groups. The default is None.
    group_col : 'list or str'
        Column to identify subset groupings in. The default is 'filename'.
    val_col : 'list or str'
     Column to preform filtering by values on. The default is 'directory'.
    
    Returns
    -------
    

### Import av data ved hjelp av DataImporter

DataImporter leser inn dataframen med metadata og bruker så denne til å lese inn og kategorisere dataene etter brukerangitte kategorier. Dataene lagres i en **dictionary**. Den bruker kun metadata_df som input argument når den instansieres. Objektet tar inn en hvilken som helst dataframe, men metodene som tilhører objektet antar at 3 kolonner eksisterer i dataframe. Dette er kolonner med:
- komplett filsti til datafilene som skal importeres
- fullt filnavn
- hvilken mappe filstiene ligger i

Objektet antar foreløpig implisitt at dataene den leter i ligger i undermapper.

#### Om metodene til DataImporter
- simple_import() importerer et sett med filer basert på en kolonne med filstier i en dataframe og returnerer en dictionary hvor datasettene ligger sortert per mappe de leses inn fra. Metoden har en mengde input argumenter som gjør den fleksibel i bruk. På det minste trenger den kun hvilken datatype den skal lese inn (filetype). For øvrige input argumenter skriv help(DataImporter.simple_import).
- add_years() legger til mappenavnet datasettet finnes til som variabel i datasettet. Dersom navnet er et år vil datasettene nå inneholde hvilken årgang de er fra.
- categorise_data() sorterer de importerte dataene basert på mappetilhørlighet, filnavn og en liste med brukerangitte kategorier. For å garantere at hver fil havner i riktig kategori er beste å bruke samme liste man brukte for å søke etter data.
    - Dersom man har mange årganger av samme datasett lagret i mange ulike subfoldere så vil **denne reorganiseringen gjøre det mulig å beregne for alle år av gangen da metoden limer alle datasett av samme kategori til en dataframe**. Hoveddtanken bak dette er å gjøre tilbakeregninger enkelt.
- year_duplicate_tidy_func() ser etter duplikatverdier i to kolonner og sletter kolonne nummer to dersom nummer en allerede finnes i datasettet. Dette er for å rette opp i der hvor add_years() er for naiv i å legge til år (f.eks for et datasett fra en folder med årsnavn men som inneholder flere årganger)
- write_data() skriver ut enten den opprinnelige dictionarien importert fra simple_import() (dersom man er usikker på om andre metoder har tullet til dataene kan dette f.eks være bra for debugging) eller den reorganiserte dictionarien laget av categorise_data()

In [4]:
# Laste inn data og få det på riktig form
data = (DataImporter(metadata_df)
        .simple_import(filetype='sas7bdat') 
        .add_years()
        .categorise_data(dataset_list=datasets)
        .year_duplicate_tidy_func(dupl_col1='aar', dupl_col2='aar_added')
        .sort_df(sort_by=['aar', 'produktkode'])
        .subset_years(df_name='energiregnskapet', start_year=2010, stop_year=2021)
        .write_data(output_object='sorted_df'))

Sorting data by category of dataset: 100%|██████████| 1/1 [00:00<00:00,  4.07it/s]


In [5]:
data['energiregnskapet'] = calculate_mineral_oil(data['energiregnskapet'])

data['energiregnskapet']

,filename,aar,produktkode,produkt_tekst,naaringskode,naaring_tekst,mengde
0,energiregnskapet_2021.sas7bdat,2010.0,EP030,Naturgass (mSm3),N.01.01.01,"Jordbruk, jakt og viltstell",2.280000e+01
46,energiregnskapet_2021.sas7bdat,2010.0,EP030,Naturgass (mSm3),H.00.00.00,Husholdninger,4.600000e+00
20,energiregnskapet_2021.sas7bdat,2010.0,EP030,Naturgass (mSm3),N.03.09.02,Produksjon av andre transportmidler,9.000000e-01
19,energiregnskapet_2021.sas7bdat,2010.0,EP030,Naturgass (mSm3),N.03.09.01,Produksjon av motorvogner og tilhengere,0.000000e+00
18,energiregnskapet_2021.sas7bdat,2010.0,EP030,Naturgass (mSm3),N.03.08.04,Produksjon av maskiner og utstyr ellers,3.300000e+00
...,...,...,...,...,...,...,...
559,energiregnskapet_2021.sas7bdat,2021.0,mineralolje,mineralolje (liter),N.08.03.04,Lagring og andre tjenester tilknyttet transport,1.265366e+08
560,energiregnskapet_2021.sas7bdat,2021.0,mineralolje,mineralolje (liter),N.09.01.00,Undervisning,1.071286e+06
561,energiregnskapet_2021.sas7bdat,2021.0,mineralolje,mineralolje (liter),N.09.02.01,Helsetjenester,2.619048e+06
562,energiregnskapet_2021.sas7bdat,2021.0,mineralolje,mineralolje (liter),N.09.02.02,"Pleie- og omsorgstjenester, barnehager og SFO",1.190476e+05


## Eksempel på bruk av funksjonene i main_program_functions.py for å lage produksjonsløpet

In [6]:
# Ta ut aktuelle subset av kildedata
jet_forb = subsetter(data, key='energiregnskapet', var='produktkode',
                     value='EP04661', dtype_convert=True, aar=int)

jet_unntak = subsetter(data, key='unntakskatalog', var='produktkode',
                       value='EP04661', dtype_convert=True, aar=int)

avgifter_jetparafin = subsetter(data, key='innbetalte_avgifter',
                                var='produktkode', value='EP04661',
                                dtype_convert=True, aar=int)

In [7]:
avgifter_jetparafin

,filename,aar,produktkode,produkt_tekst,ytart,total_avgift_kroner
4,innbetalte_avgifter_2010.sas7bdat,2010,EP04661,Jetparafin (ktonn),41364,299.0
4,innbetalte_avgifter_2011.sas7bdat,2011,EP04661,Jetparafin (ktonn),41364,298.0
4,innbetalte_avgifter_2012.sas7bdat,2012,EP04661,Jetparafin (ktonn),41364,237.0
4,innbetalte_avgifter_2013.sas7bdat,2013,EP04661,Jetparafin (ktonn),41364,219.0
4,innbetalte_avgifter_2014.sas7bdat,2014,EP04661,Jetparafin (ktonn),41364,230.0
4,innbetalte_avgifter_2015.sas7bdat,2015,EP04661,Jetparafin (ktonn),41364,458.0
4,innbetalte_avgifter_2016.sas7bdat,2016,EP04661,Jetparafin (ktonn),41364,475.0
4,innbetalte_avgifter_2017.sas7bdat,2017,EP04661,Jetparafin (ktonn),41364,476.0
4,innbetalte_avgifter_2018.sas7bdat,2018,EP04661,Jetparafin (ktonn),41364,633.0
4,innbetalte_avgifter_2019.sas7bdat,2019,EP04661,Jetparafin (ktonn),41364,551.0


In [8]:
# Test of multi_subsetter
multi_subsetter(data, key='innbetalte_avgifter', ytart='41364', produktkode='EP030')

,filename,aar,produktkode,produkt_tekst,ytart,total_avgift_kroner
8,innbetalte_avgifter_2010.sas7bdat,2010,EP030,Naturgass (mSm3),41364,30.0
8,innbetalte_avgifter_2011.sas7bdat,2011,EP030,Naturgass (mSm3),41364,79.0
8,innbetalte_avgifter_2012.sas7bdat,2012,EP030,Naturgass (mSm3),41364,83.0
8,innbetalte_avgifter_2013.sas7bdat,2013,EP030,Naturgass (mSm3),41364,118.0
8,innbetalte_avgifter_2014.sas7bdat,2014,EP030,Naturgass (mSm3),41364,124.0
8,innbetalte_avgifter_2015.sas7bdat,2015,EP030,Naturgass (mSm3),41364,101.0
8,innbetalte_avgifter_2016.sas7bdat,2016,EP030,Naturgass (mSm3),41364,113.0
8,innbetalte_avgifter_2017.sas7bdat,2017,EP030,Naturgass (mSm3),41364,130.0
8,innbetalte_avgifter_2018.sas7bdat,2018,EP030,Naturgass (mSm3),41364,321.0
8,innbetalte_avgifter_2019.sas7bdat,2019,EP030,Naturgass (mSm3),41364,293.0


Pandas har en funksjon som tillater at man kan lage en modulær pipeline slik som man gjør i R med %>% operatoren. Dette gjør at man kan bygge enkeltfunksjoner som gjør en del av jobben, og så kan man koble dem sammen slik at de til sammen kjører hele opplegget for en gitt energivare. Dette gjør at man ender opp med et modulært system hvor en selv kan sette rekkefølgen på funksjonene man bruker og hvor man selv står fritt til å definere input-argumenter for hvert trinn.

Denne metoden heter .pipe()

Kort sagt så tillater den at man kan bruke funksjoner man har brukt på egen hånd på dataframes og den lar seg kjede som alle andre pandas metoder.

In [9]:
# Alle funksjoner har lett tilgjengelig dokumentasjon
help(subsetter)

Help on function subsetter in module src.functions.main_program_functions:

subsetter(data: dict, key: 'name of dataset in dictionary as str' = None, var: str = None, value: 'float or int or str' = None, dtype_convert: bool = False, **dtype_kwargs: 'var_name = data_type')
    Function that selects a given key in the dictionary containing all
    data then a subset using df.loc to identify a column where variable
    is equal to the value defined by the user.
    
    Similar to df.loc[df[column] == value] in pandas.
    
    Parameters
    ----------
    data : Dictionary
        Dictionary containing all datasets. Dataset names = keys, datasets = 
        values. Can be access by data[key].
    key : string
        Name of dataset to subset from.
    variable : String
        Variable to filter dataset by.
    value : string or int
        Value to filter variable by.
    dtype_convert : Boolean, optional
        If set to true, function will attempt to convert variable to given 
    

### Eksempel på hvordan funksjonene kan settes sammen

In [10]:
jetparafin = (
    # Joins unntaks katalog onto forbruk by aar and næringskode
    merge_func(jet_forb, jet_unntak,
               subset_columns_r=['aar', 'naaringskode', 'unntak', 'ytart'],  # This argument selects columns 
               join_on=['aar', 'naaringskode'],
               join_method='left',
              )
    # removes filename from dataframe column
    #.pipe(filename_removal_func)
    
    # Fills in unntakskatalog
    .pipe(unntak_filler)
    
    # Calculates avgiftsbelagt mengde
    .pipe(multiplier, X='mengde', Y='unntak', new_col='avgiftsbelagt_mengde')
    
    # Calculates total avgiftsbelagt mengde
    .pipe(make_sum_column,
          group='aar',
          target_col='avgiftsbelagt_mengde',
          new_col='total_avgiftsbelagt_mengde')
    
    # Merges avgifter for jetparafin onto main dataframe
    .pipe(merge_func, avgifter_jetparafin,
          subset_columns_r=['aar', 'produktkode', 'total_avgift_kroner'],
          join_on=['aar', 'produktkode'])
    
    # Calculates avgiftsbelagt mengde
    .pipe(proportion_func,
          X='avgiftsbelagt_mengde',
          Y='total_avgiftsbelagt_mengde',
          Z='total_avgift_kroner',
          new_col='est_avgift_kroner')
    
    # Attach NR codes from omkoding to main dataframe
    .pipe(associate_codes,
         df_codes=data['omkoding_nr'],
         ind_code_col='naaringskode',
         nr_code_col = 'nr_naaring',
         prod_name_col = 'produkt_tekst',
         prod_name = 'Jetparafin')
    
    # Checks and attributes diff column to est_avgifter_kroner
    .pipe(rounding_error_dealer)

    # Reshapes an aggregates main df to nr-code level
    .pipe(column_tidy_func,
        keep_cols=['ytart', 'produkt_tekst', 'aar', 'nr_naaring'],
        avgift_col='est_avgift_kroner')
)

# Display resulting dataframe
jetparafin

,produkt,ytart,mottaker,aar,v_11,v_12,v_15,v_16
0,Jetparafin (ktonn),41364,23331,2010,0.0,0.0,4.0,0.0
1,Jetparafin (ktonn),41364,23510,2010,0.0,0.0,269.0,0.0
2,Jetparafin (ktonn),41364,23780,2010,0.0,0.0,0.0,0.0
3,Jetparafin (ktonn),41364,24842,2010,0.0,0.0,26.0,0.0
4,Jetparafin (ktonn),41364,23331,2011,0.0,0.0,0.0,0.0
5,Jetparafin (ktonn),41364,23510,2011,0.0,0.0,277.0,0.0
6,Jetparafin (ktonn),41364,23780,2011,0.0,0.0,0.0,0.0
7,Jetparafin (ktonn),41364,24842,2011,0.0,0.0,21.0,0.0
8,Jetparafin (ktonn),41364,23331,2012,0.0,0.0,0.0,0.0
9,Jetparafin (ktonn),41364,23510,2012,0.0,0.0,222.0,0.0


Når alle datasettettene er kommet på formen over kan de limes sammen vi pd.concat og deretter exporteres med batch_exporter funksjonen.

Denne funksjonen oppretter mapper basert på hvert enkelt år i datasettet og eksporterer data fra samme år i samme mappe som ett enkelt datasett.

In [11]:
# output_df = pd.concat([jetparafin, autodiesel etc...])

In [12]:
help(batch_exporter)

Help on function batch_exporter in module src.functions.export_helpers:

batch_exporter(df, year_col: str = None, export_path: str = None, prefix: str = None)
    Reads in dataframe, identifies year column and exports data to a set of
    folders that equal the amount of years in the dataframe. If these folders
    do not exist they will be created. 
    
    Data is then exported to each folder based on which year it belongs to.
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataframe for export.
    year_col : str
        Column indicating years in dataset. The default is None.
    export_path : str
        Path to subfolders in which data will be exported to. 
        The default is None.
    prefix : str
        Prefix in name of dataset. The default is None.
    
    Returns
    -------
    None.



In [13]:
#Exports datsets into subfolders
batch_exporter(jetparafin, year_col='aar', 
               export_path='../output_data', # can also be path to anywhere else on linux
               prefix='jet')

In [1]:
print('test')

NameError: name 'test' is not defined